In [137]:
from z3 import *

B = BoolSort()
Z = IntSort()
R = RealSort()
x,y,z = Reals('x y z')

def lemma(thm, by=[], admit=False):
    if not admit:
        #print(Implies(And(by), thm).sexpr())
        prove(Implies(And(by), thm))
    else:
        print("Admitted!!", thm)
    return thm
    
plus = Function("plus", R,R,R)
plus_def = ForAll([x, y], plus(x, y) == x + y)

plus_0 = lemma(ForAll([x], plus(x, 0) == x), by=[plus_def])
plus_comm = lemma(ForAll([x, y], plus(x, y) == plus(y, x)), by=[plus_def])
plus_assoc = lemma(ForAll([x, y, z], plus(x, plus(y, z)) == plus(plus(x, y), z)), by=[plus_def])
                   
mul = Function("mul", R,R,R)
mul_def = ForAll([x, y], mul(x, y) == x * y)

mul_zero = lemma(ForAll([x], mul(x, 0) == 0), by=[mul_def])
mul_1 = lemma(ForAll([x], mul(x, 1) == x), by=[mul_def])
mul_comm = lemma(ForAll([x, y], mul(x, y) == mul(y, x)), by=[mul_def])
mul_assoc = lemma(ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)), by=[mul_def])
mul_distrib = lemma(ForAll([x, y, z], mul(x, plus(y, z)) == plus(mul(x, y), mul(x, z))), by=[mul_def, plus_def])        

Nat = Datatype("Nat")
Nat.declare('zero')
Nat.declare('succ', ('pred', Nat))
Nat = Nat.create()
n,m,k = Consts('n m k', Nat)

real = Function("real", Nat, R)
real_def = ForAll([n], real(n) == If(Nat.is_zero(n), 0, plus(1, real(Nat.pred(n)))))


def induct(P):
    return Implies(And(P(Nat.zero), ForAll([n], Implies(P(n), P(Nat.succ(n))))),
                   #-----------------------------------------------------------
                   ForAll([n], P(n)))

Seq = ArraySort(Nat, R)
sum_ = Function("sum", Seq, Nat, R)
s = Const('s', Seq)
sum_def = ForAll([s, n], sum_(s, n) == If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum_(s, Nat.pred(n)))))

szero = K(Nat, RealVal(0))

#sum_const_base = lemma(sum_(szero, Nat.zero) == 0, by=[sum_def, plus_def])
#sum_const_induct = lemma(ForAll([n], Implies(sum_(szero, n) == 0, 
#                                             sum_(szero, Nat.succ(n)) == 0)),
#                         by=[sum_def, plus_def])
sum_szero = lemma(ForAll([n], sum_(szero,n) == 0), by=[sum_def, plus_def, induct(lambda n: sum_(szero, n) == 0)])


# note I original wrote ForAll([x], sum_(K(Nat, x), Nat.zero) == 0) which is not thed correct syntactic hypothesis.
# it wouldn't go through.
sum_const_base = lemma(ForAll([x], sum_(K(Nat, x), Nat.zero) == mul(real(Nat.zero), x)), 
                       by=[sum_def, mul_def, real_def])
sum_const_induct = lemma(ForAll([n], Implies(ForAll([x], sum_(K(Nat, x), n) == mul(real(n), x)), 
                                             ForAll([x], sum_(K(Nat, x), Nat.succ(n)) == mul(real(Nat.succ(n)), x)))),
                         by=[sum_def, plus_def, mul_def, real_def])
sum_const = lemma(ForAll([n, x], sum_(K(Nat, x), n) == mul(real(n), x)), 
                  by=[sum_const_base, sum_const_induct,
                      induct(lambda n: ForAll([x], sum_(K(Nat, x), n) == mul(real(n),x)))])

#sum_const_induct
#induct(lambda n: ForAll([x], sum_(K(Nat, x), n) == mul(real(n),x)))
#lemma(sum_const, by=[sum_const])

id_ = Lambda([n], real(n))
# helper function. We could perhaps have4 lambdified the sympy result?
nn1 = lambda n: mul(real(n) - 1, real(n)) / 2 

sum_n_base = lemma(ForAll([n], sum_(id_, Nat.zero) == nn1(Nat.zero)), by=[sum_def, plus_def, mul_def, real_def])
sum_n_induct = lemma(ForAll([n], Implies(sum_(id_, n) == nn1(n), 
                                        sum_(id_, Nat.succ(n)) == nn1(Nat.succ(n)))),            
                     by=[sum_def, plus_def, mul_def, real_def])

sum_n = lemma(ForAll([n], sum_(id_, n) == nn1(n)), by=[sum_n_base, sum_n_induct, induct(lambda n: sum_(id_, n) == nn1(n))])

proved
proved
proved
proved
proved
proved
proved
proved
proved
proved
proved
proved
Array(Nat, Real)
proved
proved
proved


In [122]:
import sympy as sp
import sympy.abc as abc

# sum is inclusive of boundaries.
sp.Sum(abc.n, (abc.n, 0, abc.m-1)).doit()
#print(sp.Sum(1, (abc.n, 0, 1)).doit())

m**2/2 - m/2

We expect it to often be the case that sympy closed forms will be confirmable via a rote induction procedure.

In [138]:
# https://en.wikipedia.org/wiki/Cauchy_sequence
cauchy = Function("cauchy", Seq, B)
eps = Real('eps')
N = Const("N", Nat)
cauchy_def = ForAll([s], cauchy(s) == 
                    ForAll([eps], Exists([N], ForAll([n, m], Implies(And(n >= N, m >= N), abs(s[n] - s[m]) < eps)))))



SyntaxError: incomplete input (589912192.py, line 5)

In [ ]:
mono = Function("mono", Seq, B)
mono_def = ForAll([s], mono(s) == ForAll([n], s[n] <= s[Nat.succ(n)]))

bounded = Function("bounded", Seq, B)
bounded_def = ForAll([s], bounded(s) == Exists([y], ForAll([n], s[n] <= y)))

In [ ]:
RFun = ArraySort(R,R)
cont = Function(RFun, B)

cont_def = ForAll([f], cont(f) == ForAll([eps], Exists([N], ForAll([n], Implies(n >= N, abs(f(n) - f(0)) < eps)))))


In [105]:
# RecFunction allows us to use z3 as a simplifier.

#sum__ = RecFunction("sum", Seq, Nat, R)
#RecAddDefinition(sum__, [s, n], If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum__(s, Nat.pred(n)))))
#simplify(sum__(Lambda([n], If(Nat.is_zero(n), RealVal(0), RealVal(1))), Nat.succ(Nat.zero)))

#sum_id = RecFunction("sum_id", Seq, Nat, R)
#RecAddDefinition(sum_id, [n], If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum__(s, Nat.pred(n)))))
#simplify(sum__(Lambda([n], If(Nat.is_zero(n), RealVal(0), RealVal(1))), Nat.succ(Nat.zero)))

real_ = RecFunction("real", Nat, R)
RecAddDefinition(real_, [n], If(Nat.is_zero(n), 0, 1 + real_(Nat.pred(n))))
print(simplify(real_(Nat.succ(Nat.succ(Nat.zero)))))
simplify(real_(Nat.succ(Nat.succ(n))))


2


real(succ(succ(n)))

In [71]:
%%file /tmp/induct.smt2
(declare-sort Nat 0)
(declare-fun sum ((Array Nat Real) Nat) Real)
(declare-fun real (Nat) Real)
(declare-fun plus (Real Real) Real)
(declare-fun mul (Real Real) Real)
(assert (let ((a!1 (forall ((x Real))
             (= (sum ((as const (Array Nat Real)) x) zero) 0.0)))
      (a!2 (forall ((n Nat))
             (let ((a!1 (forall ((x Real))
                          (= (sum ((as const (Array Nat Real)) x) n)
                             (mul (real n) x))))
                   (a!2 (forall ((x Real))
                          (= (sum ((as const (Array Nat Real)) x) (succ n))
                             (mul (real (succ n)) x)))))
               (=> a!1 a!2))))
      (a!3 (forall ((x Real))
             (= (sum ((as const (Array Nat Real)) x) zero) (mul (real zero) x))))
      (a!4 (forall ((n Nat))
             (forall ((x Real))
               (= (sum ((as const (Array Nat Real)) x) n) (mul (real n) x)))))
      (a!5 (forall ((x Real) (n Nat))
             (= (sum ((as const (Array Nat Real)) x) n) (mul (real n) x)))))
  (=> (and a!1 a!2 (=> (and a!3 a!2) a!4)) a!5)))

(check-sat)

Overwriting /tmp/induct.smt2


In [70]:
! vampire /tmp/induct.smt2

% Running in auto input_syntax mode. Trying SMTLIB2
% Failed with
% User error: Compound functor expected to be a rankend function (starting with '_'). Instead read: (as const (Array Nat Real))
% Trying TPTP
Parsing Error on line 1: cnf(), fof(), vampire() or include() expected at position 0 (text: ()
